In [1]:
import os
import json
import pandas as pd
import tensorflow as tf
from datetime import datetime as dt

from hmc.utils.dir import create_dir
from hmc.model.arguments import  build

2024-06-01 17:49:25.276987: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 17:49:26.186082: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from tqdm.notebook import tqdm

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
import os
from datetime import datetime

def horario_atual_para_diretorio():
    # Obtém a data e hora atual
    data_hora_atual = datetime.now()
    
    # Formata a data e hora para o formato de diretório desejado
    diretorio = data_hora_atual.strftime('%Y_%m_%d_%H_%M_%S')
    
    return diretorio

# Exemplo de uso
diretorio_formatado = horario_atual_para_diretorio()

In [5]:
# Set python level verbosity
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.DEBUG)

# Set C++ Graph Execution level verbosity
os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(tf.compat.v1.logging.DEBUG)

base_path = "/mnt/disks/data/fma/trains"
sample_id = "hierarchical_tworoots_dev"


train_path = os.path.join(base_path, sample_id)
tfrecords_path =os.path.join(train_path,'tfrecords')
metadata_path = os.path.join(train_path,"metadata.json")
labels_path = os.path.join(train_path,"labels.json")

model_id = diretorio_formatado

model_path = os.path.join(train_path, model_id)

args = pd.Series({
    "batch_size":32,
    "epochs":10,
    "dropout":0.3,
    'patience':1,
    'max_queue_size':64,
    "labels_path": labels_path,
    "metadata_path": metadata_path,
    "trainset_pattern": os.path.join(tfrecords_path,'train'),
    "testset_pattern": os.path.join(tfrecords_path,'test'),
    "valset_pattern": os.path.join(tfrecords_path,'val'),
    "model_path":model_path
})


In [6]:
create_dir(model_path)

True

In [7]:

import multiprocessing


BUFFER_SIZE = 10


class Dataset:
    def __init__(self,files,epochs, batch_size, levels_size):
        self.epochs = epochs
        self.batch_size = batch_size
        self.files = files
        self.depth = len(levels_size)
        self.levels_size = levels_size

    def load_dataframe(self, dataset):
        columns = ['features','labels']
        
        
        df = pd.DataFrame(
            dataset.as_numpy_iterator(),
            columns=columns
        )
    
        df.dropna(inplace=True)
    
        return df

    def build(self,df=False):
        
        files = [os.path.join(self.files,file) for file in os.listdir(self.files)]

        ds = tf.data.TFRecordDataset(files)
        
        '''''
            Shuffle and reapeat
        '''''
        
        #ds = ds.shuffle(buffer_size=1024 * 50 * 10)
        #ds = ds.repeat(count=self.epochs)
        
        
        '''''
            Map and batch
        '''''
        
                      
        ds = ds.map(self.__parse__, num_parallel_calls=None)

        if df==True:
            return self.load_dataframe(ds)
        
        ds = ds.batch(self.batch_size,drop_remainder=False)
        
        
                      
        ds = ds.prefetch(buffer_size=5)
        
        
        return ds

    def convert_to_binary(self, label, num_classes):
        # Filtrar índices negativos
        valid_indices = tf.boolean_mask(label, label >= 0)
        
        # Inicializar vetor binário
        binary_label = tf.zeros(num_classes, dtype=tf.float32)
        
        # Atualizar vetor binário com base nos rótulos válidos
        indices = tf.expand_dims(valid_indices, 1)
        updates = tf.ones_like(valid_indices, dtype=tf.float32)
        binary_label = tf.tensor_scatter_nd_update(binary_label, indices, updates)

        return binary_label

    def __parse__(self, element):
        data = {}
        for level in range(1, self.depth+1):
            data[f'label{level}'] = tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True)
        
        data.update({
            'features': tf.io.FixedLenFeature([1280], tf.float32),
            'track_id' : tf.io.FixedLenFeature([], tf.int64),
        })
        
        content = tf.io.parse_single_example(element, data)

        labels = {}
        for level in range(1, self.depth+1):
            local_label = content[f'label{level}']
            binary_label = self.convert_to_binary(local_label, self.levels_size[f'level{level}'])
            labels.update({f'level{level}_output': binary_label})
        

        inp = {"features":content['features'] }

        return inp, labels


In [21]:
from keras import layers
from keras.layers import Dense, Dropout, Reshape, Concatenate, Flatten, Input, Normalization
from keras.optimizers import Adam

import numpy as np

# class OutputNormalization(layers.Layer):
#     def call(self, x, **kwargs):
#         norm_output = tf.one_hot(tf.math.argmax(x, axis=1), x.shape[1], dtype=x.dtype)
#         norm_output = Flatten()(norm_output)
#         norm_output = Reshape((1,))(norm_output)
#         return norm_output

#     def compute_output_shape(self, input_shape):
#         return input_shape

class OutputNormalization(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.nn.softmax(inputs)  # Exemplo de normalização, ajuste conforme necessário


def _load_weights(model, weights_path):
    print(model.summary())

    if weights_path and tf.io.gfile.exists(f"{weights_path}.index"):
        print(f"load weights for model {model.name}. weights_path={weights_path}")
        model.load_weights(weights_path)

    return model


def build_cnn(feature, input_shape):
    x: object = Normalization(input_shape=[input_shape, 1], axis=None)(feature)
    x = layers.Conv1D(128, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(64, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Conv1D(32, 3, activation='relu', padding="valid")(x)
    x = layers.MaxPooling1D()(x)
    x = layers.Flatten()(x)

    return x


def build_classification(x, size, dropout, input_shape=1024, name='level1'):
    #x: object = Dense(input_shape, activation='relu')(x)
    #x = Dropout(dropout)(x)
    #x = Dense(int(input_shape/2), activation='relu')(x)
    #x = Dropout(dropout)(x)
    x = Dense(int(input_shape/2), activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(size, activation='sigmoid', name=name+'_output')(x)

    return x


def build_model(levels_size: dict, sequence_size: int = 1280, dropout: float = 0.7) -> tf.keras.models.Model:
    """

    :rtype: tf.keras.models.Model
    """
    input_shape = (sequence_size, 1)
    fcn_size = 1024
    input_layer = Input(shape=input_shape, dtype=tf.float32, name="features")
    x = Dense(fcn_size, activation='relu')(input_layer)
    x = Dropout(dropout)(x)
    x = Dense(int(fcn_size / 2), activation='relu')(x)
    x = Dropout(dropout)(x)
    #x_flat = Flatten()(input_layer)
    
    depth = len(levels_size)

    #x: object = build_cnn(music, input_shape)

    # Construção das camadas sequencialmente
    prev_output = x
    outputs = []
    for level, size in levels_size.items():
        # Construir a camada atual
        current_input = prev_output 
        #current_input = prev_output if level == 'level1' else Concatenate(axis=1)([Flatten()(OutputNormalization()(prev_output)), input_layer])
        output = build_classification(current_input, size, dropout,
                                              input_shape=fcn_size, name=level)

        # Atualizar a saída anterior para a próxima iteração
        prev_output = output
        outputs.append(prev_output)
    

    model = tf.keras.models.Model(inputs=input_layer, outputs=outputs, name="Essentia")

    #     _load_weights(model, weights_path)
    
    optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(optimizer=optimizer,
                   loss='binary_crossentropy', metrics=['accuracy']*4)

    return model


In [22]:
with open(args.metadata_path, 'r') as f:
    metadata = json.loads(f.read())
    print(metadata)

{'sequence_size': 1280, 'max_depth': 4, 'levels_size': [2, 30, 16], 'val_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/val', 'train_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/train', 'test_path': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/tfrecords/test', 'val_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/val.csv', 'train_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/train.csv', 'test_csv': '/mnt/disks/data/fma/trains/hierarchical_tworoots_dev/test.csv', 'trainset_count': 16791, 'validationset_count': 2007, 'testset_count': 4814}


In [23]:
metadata['trainset_count']

16791

In [24]:
with open(args.labels_path, 'r') as f:
    labels = json.loads(f.read())

In [25]:
levels_size = {'level1': labels['label_1_count'] ,
                   'level2': labels['label_2_count'] ,
                   'level3': labels['label_3_count'] ,
                   'level4': labels['label_4_count'] }

In [27]:
for level, labels in levels_size.items():
    print(level)

level1
level2
level3
level4


In [14]:
depth = len(levels_size)

In [15]:
#ds_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=False)

In [16]:
#df_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, levels_size).build(df=True)

In [18]:
from keras.callbacks import EarlyStopping
# from sabotage.model.callback import ValidateCallback, BackupAndRestoreCheckpoint

def run(args: object):
    print(args)

    with open(args.metadata_path, 'r') as f:
        metadata = json.loads(f.read())
        print(metadata)

    with open(args.labels_path, 'r') as f:
        labels = json.loads(f.read())

    levels_size = {'level1': labels['label_1_count'] ,
                   'level2': labels['label_2_count'] ,
                   'level3': labels['label_3_count'] ,
                   'level4': labels['label_4_count'] }

    params: dict = {
        'levels_size': levels_size,
        'sequence_size': metadata['sequence_size'],
        'dropout': args.dropout
    }

    print(params)
    model = build_model(**params)
    print(model.summary())
    
    tf.keras.utils.plot_model(
        model,
        to_file="model.png",
        show_shapes=False,
        show_dtype=False,
        show_layer_names=True,
        rankdir="TB",
        expand_nested=False,
        dpi=96,
        layer_range=None,
        show_layer_activations=False,
        show_trainable=False,
    )

    depth = len(params['levels_size'])
    
    ds_train = Dataset(args.trainset_pattern, args.epochs, args.batch_size, params['levels_size']).build(df=False)
    ds_validation = Dataset(args.valset_pattern, args.epochs, args.batch_size, params['levels_size']).build(df=False)
    callbacks = [EarlyStopping(monitor='loss', patience=args.patience, verbose=1)]
    model.fit(ds_train,
              validation_data=ds_validation,
              steps_per_epoch=metadata['trainset_count'] // args.batch_size,
              validation_steps=metadata['validationset_count'] // args.batch_size,
              epochs=args.epochs,
              callbacks=callbacks)

    model.save(os.path.join(args.model_path, 'best_binary.keras'))


In [168]:

time_start = dt.utcnow()
print("[{}] Experiment started at {}".format(id, time_start.strftime("%H:%M:%S")))
print(".......................................")
print(args)
run(args)
time_end = dt.utcnow()
time_elapsed = time_end - time_start
print(".......................................")
print("[{}] Experiment finished at {} / elapsed time {}s".format(id, time_end.strftime("%H:%M:%S"), time_elapsed.total_seconds()))

[<built-in function id>] Experiment started at 01:11:34
.......................................
batch_size                                                         32
epochs                                                             10
dropout                                                           0.3
patience                                                            1
max_queue_size                                                     64
labels_path         /mnt/disks/data/fma/trains/hierarchical_tworoo...
metadata_path       /mnt/disks/data/fma/trains/hierarchical_tworoo...
trainset_pattern    /mnt/disks/data/fma/trains/hierarchical_tworoo...
testset_pattern     /mnt/disks/data/fma/trains/hierarchical_tworoo...
valset_pattern      /mnt/disks/data/fma/trains/hierarchical_tworoo...
model_path          /mnt/disks/data/fma/trains/hierarchical_tworoo...
dtype: object
batch_size                                                         32
epochs                                            

Model: "Essentia"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ features (InputLayer)           │ (None, 1280, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 1280, 512)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_67 (Dropout)            │ (None, 1280, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ level1_output (Dense)           │ (None, 1280, 2)        │         1,026 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 1280, 512)      │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_68 (Dropout)            │ (None, 1280, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ level2_output (Dense)           │ (None, 1280, 30)       │        15,390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 1280, 512)      │        15,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_69 (Dropout)            │ (None, 1280, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ level3_output (Dense)           │ (None, 1280, 16)       │         8,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 1280, 512)      │         8,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_70 (Dropout)            │ (None, 1280, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ level4_output (Dense)           │ (None, 1280, 3)        │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,299 (208.20 KB)

 Trainable params: 53,299 (208.20 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 2), output.shape=(None, 1280, 2)